In [1]:
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv"
df = pd.read_csv(url)

# Display the first few rows to understand the data structure
df.head()


,Customer,ST,GENDER,Education,Customer Lifetime Value,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Vehicle Class,Total Claim Amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323


## Challenge 1: Data Cleaning and Formatting


In [2]:
# Cleaning columns names
# Step 1: Standardize column names
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('st', 'state')

# Display updated column names
print("Updated Column Names:", df.columns.tolist())


Updated Column Names: ['custateomer', 'state', 'gender', 'education', 'custateomer_lifetime_value', 'income', 'monthly_premium_auto', 'number_of_open_complaints', 'policy_type', 'vehicle_class', 'total_claim_amount']


In [4]:
# Exercise 2: Cleaning invalid values
# Step 2: Clean "gender" column
df['gender'] = df['gender'].str.upper().replace({
    "FEMALE": "F", "FEMAL": "F",
    "MALE": "M", "M": "M"
})

# Step 3: Clean "state" column
state_mapping = {
    "AZ": "Arizona", "Cali": "California", "WA": "Washington"
}
df['state'] = df['state'].replace(state_mapping)

# Step 4: Clean "education" column
df['education'] = df['education'].replace({"Bachelors": "Bachelor"})

# Step 5: Remove '%' from "customer_lifetime_value" and convert to numeric
df['custateomer_lifetime_value'] = df['custateomer_lifetime_value'].str.replace('%', '').astype(float)

# Step 6: Simplify "vehicle_class" column
df['vehicle_class'] = df['vehicle_class'].replace({
    "Sports Car": "Luxury", "Luxury SUV": "Luxury", "Luxury Car": "Luxury"
})

# Display cleaned data
df.head()


,custateomer,state,gender,education,custateomer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323


In [5]:
# Exercise 3: Formatting data types
# Convert "number_of_open_complaints" to a numeric column by extracting the middle number
df['number_of_open_complaints'] = df['number_of_open_complaints'].str.split('/').str[1].astype(float)

# Verify the updated data types
df.dtypes


,0
custateomer,object
state,object
gender,object
education,object
custateomer_lifetime_value,float64
income,float64
monthly_premium_auto,float64
number_of_open_complaints,float64
policy_type,object
vehicle_class,object


In [6]:
# Exercise 4: Dealing with null values
# Step 1: Check for null values
null_counts = df.isnull().sum()
print("Null values per column:\n", null_counts)

# Step 2: Fill null values

for column in df.select_dtypes(include=['float64', 'int64']).columns:
    df[column].fillna(df[column].median(), inplace=True)

for column in df.select_dtypes(include=['object']).columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

# Verify if there are still null values
df.isnull().sum()


Null values per column:
 custateomer                   2937
state                         2937
gender                        3054
education                     2937
custateomer_lifetime_value    2940
income                        2937
monthly_premium_auto          2937
number_of_open_complaints     2937
policy_type                   2937
vehicle_class                 2937
total_claim_amount            2937
dtype: int64


<ipython-input-6-4e055d757dc5>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].median(), inplace=True)
<ipython-input-6-4e055d757dc5>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

,0
custateomer,0
state,0
gender,0
education,0
custateomer_lifetime_value,0
income,0
monthly_premium_auto,0
number_of_open_complaints,0
policy_type,0
vehicle_class,0


In [7]:
# Exercise 5: Dealing with duplicates
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Remove duplicates and reset index
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

# Verify if duplicates are removed
df.duplicated().sum()


Number of duplicate rows: 2936


0

## Challenge 2: Creating functions in a separte py file

In [8]:
# To save this in a file named data_cleaning.py

import pandas as pd

def clean_column_names(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('st', 'state')
    return df

def clean_gender(df):
    df['gender'] = df['gender'].str.upper().replace({
        "FEMALE": "F", "FEMAL": "F", "MALE": "M"
    })
    return df

def clean_state(df):
    state_mapping = {"AZ": "Arizona", "Cali": "California", "WA": "Washington"}
    df['state'] = df['state'].replace(state_mapping)
    return df

def clean_education(df):
    df['education'] = df['education'].replace({"Bachelors": "Bachelor"})
    return df

def clean_clv(df):
    df['customer_lifetime_value'] = df['customer_lifetime_value'].str.replace('%', '').astype(float)
    return df

def clean_vehicle_class(df):
    df['vehicle_class'] = df['vehicle_class'].replace({
        "Sports Car": "Luxury", "Luxury SUV": "Luxury", "Luxury Car": "Luxury"
    })
    return df

def handle_missing_values(df):
    for column in df.select_dtypes(include=['float64', 'int64']).columns:
        df[column].fillna(df[column].median(), inplace=True)
    for column in df.select_dtypes(include=['object']).columns:
        df[column].fillna(df[column].mode()[0], inplace=True)
    return df

def remove_duplicates(df):
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df


## Challenge 3: Analyzing Clean Data

In [10]:
# Calculate the 75th percentile for Total Claim Amount
claim_75th_percentile = df['total_claim_amount'].quantile(0.75)

# Calculate the 25th percentile for Customer Lifetime Value
clv_25th_percentile = df['custateomer_lifetime_value'].quantile(0.25)

# Filter data for high claim amount and low customer lifetime value
high_claim_low_clv = df[
    (df['total_claim_amount'] > claim_75th_percentile) &
    (df['custateomer_lifetime_value'] < clv_25th_percentile)
]

# Display summary statistics for this group
summary = high_claim_low_clv[['total_claim_amount', 'custateomer_lifetime_value']].describe()
summary


,total_claim_amount,custateomer_lifetime_value
count,40.000000,40.000000
mean,723.776577,341577.910000
std,149.280606,56921.472027
min,537.600000,228759.690000
25%,633.378846,304220.875000
50%,685.284189,365995.370000
75%,799.200000,387364.700000
max,1185.988301,402381.440000


In [11]:
## Ending saving the data

df.to_csv('cleaned_insurance_data.csv', index=False)
